In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperForConditionalGeneration, WhisperProcessor, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers.utils import logging
from datasets import load_from_disk
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch
import evaluate
import soundfile
from pathlib import Path
from transformers.onnx import export

In [4]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]
    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [6]:
train_dataset = load_from_disk(dataset_path=r"C:\Users\Vijay\OneDrive\Documents\GitHub\til-24-overflow\hf_datasets\train.hf")
val_dataset = load_from_disk(dataset_path=r"C:\Users\Vijay\OneDrive\Documents\GitHub\til-24-overflow\hf_datasets\val.hf")
train_dataset_mini = load_from_disk(dataset_path=r"C:\Users\Vijay\OneDrive\Documents\GitHub\til-24-overflow\hf_datasets\train_mini.hf")
val_dataset_mini = load_from_disk(dataset_path=r"C:\Users\Vijay\OneDrive\Documents\GitHub\til-24-overflow\hf_datasets\val_mini.hf")

In [7]:
train_dataset = train_dataset.map(prepare_dataset)
val_dataset = val_dataset.map(prepare_dataset)
train_dataset_mini = train_dataset_mini.map(prepare_dataset)
val_dataset_mini = val_dataset_mini.map(prepare_dataset)

Map:   0%|          | 0/2800 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

In [8]:
# model = WhisperForConditionalGeneration.from_pretrained("asr_training/checkpoint-175")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")
model.generation_config.language = "english"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

C:\Users\Vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Vijay\.cache\huggingface\hub\models--openai--whisper-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

In [9]:
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="English", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [11]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [12]:
metric = evaluate.load("wer")

In [13]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [17]:
training_args = Seq2SeqTrainingArguments(
    output_dir="asr_training_small",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    # warmup_steps=500,
    # max_steps=5000,
    num_train_epochs=4,
    gradient_checkpointing=True,
    fp16=True,
    # evaluation_strategy="steps",
    evaluation_strategy="epoch",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    # save_steps=1000,
    # eval_steps=1000,
    # logging_steps=25,
    logging_strategy="epoch",
    save_strategy="epoch",
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [18]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [20]:
trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


CommitInfo(commit_url='https://huggingface.co/Shadowmachete/asr_training_small/commit/b297bf71a31e0f3e4fba6871c9ad4115166a03ef', commit_message='End of training', commit_description='', oid='b297bf71a31e0f3e4fba6871c9ad4115166a03ef', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
# trainer.train(resume_from_checkpoint=r"C:\Users\Vijay\OneDrive\Documents\GitHub\til-24-overflow\asr_training\checkpoint-175")
trainer.train()

  0%|          | 0/700 [00:00<?, ?it/s]

{'loss': 0.1348, 'grad_norm': 3.0882246494293213, 'learning_rate': 7.52857142857143e-06, 'epoch': 1.0}


  0%|          | 0/88 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


{'eval_loss': 0.04688960313796997, 'eval_wer': 5.126254943717676, 'eval_runtime': 181.7031, 'eval_samples_per_second': 3.852, 'eval_steps_per_second': 0.484, 'epoch': 1.0}


...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


{'eval_loss': 0.04179147258400917, 'eval_wer': 3.7952540310313356, 'eval_runtime': 185.7953, 'eval_samples_per_second': 3.768, 'eval_steps_per_second': 0.474, 'epoch': 4.0}


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


{'train_runtime': 2578.2487, 'train_samples_per_second': 4.344, 'train_steps_per_second': 0.272, 'train_loss': 0.04873574733734131, 'epoch': 4.0}


TrainOutput(global_step=700, training_loss=0.04873574733734131, metrics={'train_runtime': 2578.2487, 'train_samples_per_second': 4.344, 'train_steps_per_second': 0.272, 'total_flos': 7.26431956992e+17, 'train_loss': 0.04873574733734131, 'epoch': 4.0})